<a href="https://colab.research.google.com/github/sac-marius/colab-test/blob/main/solana_blocks_parallel_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔗 Solana blokkanalyse – parallell uthenting via RPC

Dette prosjektet ble satt opp med ChatGPT 1. august 2025. Samtalen kan refereres her: [https://chatgpt.com/share/e/688d1ed5-04f0-800f-a579-80b2a559e77c](https://chatgpt.com/share/e/688d1ed5-04f0-800f-a579-80b2a559e77c)

GitHub-bruker: `sac-marius`

## Innhold:
- Henter de siste 1000 blokkene fra Solana
- Bruker 10 samtidige tråder (`ThreadPoolExecutor`)
- Parser ut transaksjonsdata: `timestamp`, `slot`, `from/to`, `lamports`
- Lagrer som komprimert `.parquet`

---


In [1]:
!pip install solana pandas pyarrow tqdm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 64.0 MB/s eta 0:00:00
  Created wheel for construct: filename=construct-2.10.68-py3-none-any.whl size=59223 sha256=a043b491743d0f86673b28438beb43f9ac34865210251789cdcebd8317d8e670
  Stored in directory: /root/.cache/pip/wheels/6a/5b/a1/35e70b419451f0f619898c0f9ec10f3d920721daed7bc24eab
Successfully built construct


In [2]:
from solana.rpc.api import Client
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
from tqdm import tqdm
import time

client = Client("https://api.mainnet-beta.solana.com")


In [6]:
from solana.rpc.api import Client
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
from tqdm import tqdm
import time

client = Client("https://api.mainnet-beta.solana.com")

def get_latest_slot():
    response = client.get_slot()
    return response.value

def get_block_data(slot):
    try:
        response = client.get_block(slot, max_supported_transaction_version=0)
        result = response['result']
        if not result:
            return []

        block_time = result['blockTime']
        txs = result['transactions']
        parsed = []
        for tx in txs:
            message = tx['transaction']['message']
            meta = tx['meta']
            if not meta or not meta.get('postBalances'):
                continue

            accounts = message['accountKeys']
            from_addr = accounts[0]
            to_addr = accounts[1] if len(accounts) > 1 else None

            lamports = (
                meta['postBalances'][1] - meta['preBalances'][1]
                if len(meta['postBalances']) > 1 else 0
            )

            parsed.append({
                'slot': slot,
                'timestamp': block_time,
                'tx_signature': tx['transaction']['signatures'][0],
                'from_address': from_addr,
                'to_address': to_addr,
                'lamports': lamports
            })
        return parsed

    except Exception as e:
        print(f"⚠️ Feil på slot {slot}: {e}")
        return []

In [7]:
start_time = time.time()

latest_slot = get_latest_slot()
n = 1000
slots = list(range(latest_slot - n, latest_slot))

results = []
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = {executor.submit(get_block_data, slot): slot for slot in slots}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Henter blokker"):
        data = future.result()
        results.extend(data)

df = pd.DataFrame(results)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

df.to_parquet("solana_blocks.parquet", index=False, compression='snappy')

total_time = time.time() - start_time
print("✅ Lagret som 'solana_blocks.parquet'")
print(f"⏱️ Total tid: {total_time:.2f} sekunder for {n} blokker")

Henter blokker:   0%|          | 1/1000 [00:00<02:23,  6.95it/s]

⚠️ Feil på slot 357230918: 
⚠️ Feil på slot 357230916: 
⚠️ Feil på slot 357230919: 
⚠️ Feil på slot 357230920: 
⚠️ Feil på slot 357230921: 


Henter blokker:   1%|          | 6/1000 [00:00<00:53, 18.49it/s]

⚠️ Feil på slot 357230922: ⚠️ Feil på slot 357230923: 

⚠️ Feil på slot 357230926: 
⚠️ Feil på slot 357230925: 
⚠️ Feil på slot 357230924: 


Henter blokker:   1%|          | 11/1000 [00:00<00:46, 21.09it/s]

...

Henter blokker:  98%|█████████▊| 985/1000 [00:21<00:00, 45.88it/s]

⚠️ Feil på slot 357231891: ⚠️ Feil på slot 357231892: 

⚠️ Feil på slot 357231894: 
⚠️ Feil på slot 357231893: 
⚠️ Feil på slot 357231895: 
⚠️ Feil på slot 357231897: 
⚠️ Feil på slot 357231896: 
⚠️ Feil på slot 357231898: 
⚠️ Feil på slot 357231899: 
⚠️ Feil på slot 357231900: 


Henter blokker: 100%|██████████| 1000/1000 [00:21<00:00, 45.77it/s]

⚠️ Feil på slot 357231903: 
⚠️ Feil på slot 357231901: 
⚠️ Feil på slot 357231904: 
⚠️ Feil på slot 357231902: 
⚠️ Feil på slot 357231905: 
⚠️ Feil på slot 357231906: 
⚠️ Feil på slot 357231907: 
⚠️ Feil på slot 357231908: 
⚠️ Feil på slot 357231909: 
⚠️ Feil på slot 357231910: 
⚠️ Feil på slot 357231911: 


KeyError: 'timestamp'

In [5]:
response = client.get_slot()
print(type(response))
print(dir(response))

<class 'solders.rpc.responses.GetSlotResp'>
['__bytes__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'from_bytes', 'from_json', 'to_json', 'value']


In [ ]:
slot_to_inspect = latest_slot - 10  # Inspect a block that was likely attempted in the previous run
try:
    response = client.get_block(slot_to_inspect, max_supported_transaction_version=0)
    print(type(response))
    print(response)
except Exception as e:
    print(f"Error fetching block {slot_to_inspect}: {e}")